## 과제명: 기간별 강의의뢰서 첨부파일 다운로드 자동화

### T1 : 정보 입력

- 패키지 import

In [ ]:
import pyautogui as pg
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup as bs4
import pandas as pd

- EP 계정 정보 입력(ID, PASS)

In [ ]:
my_id = 'user_id'
my_pw = pg.password(title='EP_PASS', text='패스워드를 입력하세요', default='')

- 검색 정보입력
   - 검색 키워드: '강의의뢰서'
   - 검색 기간 : 현재 ~ 필요일자(형식:'2022-11-01')

In [ ]:
search_key = '강의의뢰서'
search_date = '2022-11-20'

### T2 : EP 접속 및 로그인

- EP 접속

In [ ]:
url = 'http://swpsso.posco.net/idms/U61/jsp/login/login.jsp'

driver = webdriver.Edge('d:/작업방/msedgedriver')
driver.get(url)
driver.implicitly_wait(10) #드라이버가 로딩되는데 10초까지 대기, 빨리 로딩되면 다음 코드 진행
driver.maximize_window()

- 계정 입력

In [ ]:
driver.find_element(By.XPATH,'//*[@id="username"]').send_keys(my_id)
driver.find_element(By.XPATH,'//*[@id="password"]').send_keys(my_pw)
driver.find_element(By.XPATH,'//*[@id="loginsubmit"]').click()


- 예외처리 : 기존 로그인 경고창이 열릴 경우 alert accept 처리

In [ ]:
pg.sleep(5)
try:  
    alert=driver.switch_to.alert #to_alert --> _는 구버전으로 크롬에서 안됨
    alert.accept()
except:
    print("기존 로그인 경고창 없음. 계속 진행.")  # 경고창이 열리지 않을 경우

- 예외처리 : 현재 활성화된 드라이버 창 확인하여 SSO Notice 창 닫기 

In [ ]:
pg.sleep(2) #슬립이 없으면 에러

try:
      for i in reversed(range(len(driver.window_handles))):
            driver.switch_to.window(driver.window_handles[i])
            if driver.title != 'EP(Enterprise Portal)':
                  driver.close()
except:
    print("팝업창 없음-계속 진행")  # 팝업창이 하나도 없을 경우

### T3 : 메일 검색

- 메일 화면 열기

In [ ]:
pg.sleep(5) #로그인 되는 시간을 충분히 줌

driver.switch_to.window(driver.window_handles[-1]) #최근창으로 전환
driver.find_element(By.XPATH,"//*[@id='533982']").click()

- 메일 검색 : 강의의뢰서

In [ ]:
pg.sleep(3)

driver.switch_to.window(driver.window_handles[-1]) #최근창으로 전환
driver.find_element(By.XPATH,'//*[@id="Query"]').send_keys(search_key)
driver.find_element(By.XPATH,'//*[@id="Query"]').send_keys(Keys.ENTER)

- 메일 목록 추출

In [ ]:
pg.sleep(1)
soup = bs4(driver.page_source, "html.parser")
#테이블 가져오기
table = soup.select('#dis') #테이블에 id 값이 없어서 상위의 div의 id갑 사용
table_html = str(table) #판다스로 HTML을 읽기 위해 str으로 변환
df = pd.read_html(table_html) #판다스로 HTML 읽기
df_01 = df[0] #데이터 프레임이 리스트 안에 있어서 0번만 가져옴
print(df_01)

- 기간 내 데이터 추출

In [ ]:
pg.sleep(1)
df_02 = df_01.loc[(df_01['날짜'] >= search_date) | (df_01['날짜'] <= '1900.01.01')] #현재일은 시간만 나오므로 1900년보다 작은 것으로 검색
print(df_02)

- 최종 추출 목록 count

In [ ]:
mail_cnt = len(df_02)
mail_cnt 

### T4 : 첨부파일 다운로드

- 메일 제목 엘리먼트 선택

In [ ]:
select = '#dis tr > td:nth-child(7) > a:nth-child(2)' #새창보기로 선택
titles = driver.find_elements(By.CSS_SELECTOR, select)

- 제목 클릭 및 첨부파일 다운로드

In [ ]:
for i in range(0, mail_cnt):
      titles[i].click() #메일제목 클릭
      pg.sleep(2)
      driver.switch_to.window(driver.window_handles[-1]) #최근창으로 전환

      try:
          driver.find_element(By.XPATH,'//*[@id="download_all"]').click() #첨부 다운로드 클릭
          pg.sleep(3)
          #다운로드 허용 클릭(첫번째만 동작)
          btn_pass = pg.locateCenterOnScreen('btn_pass.png')
          if btn_pass != None: pg.click(btn_pass)
      except:
          print("첨부파일이 없으면 다음으로 넘어가세요")  # 첨부파일이 없을 경우

      pg.sleep(5) #다운 받는 시간 필요   
      driver.close()
      driver.switch_to.window(driver.window_handles[-1]) #최근창으로 전환
      pg.sleep(1)